In [ ]:
import requests
import os
from dotenv import load_dotenv
import pandas as pd
# Load API key from .env
load_dotenv()
EIA_API_KEY = os.getenv("EIA_API_KEY")

def get_eia_data(series_id):
    url = f"https://api.eia.gov/v2/seriesid/{series_id}?api_key={EIA_API_KEY}"
    r = requests.get(url)
    r.raise_for_status()
    data = r.json()["response"]["data"]
    df = pd.DataFrame(data)
    df["date"] = pd.to_datetime(df["period"])
    df["value"] = pd.to_numeric(df["value"], errors="coerce")
    return df[["date", "value"]].sort_values("date")




HTTPError: 500 Server Error: Internal Server Error for url: https://api.eia.gov/v2/seriesid/PET.MCRFPW.M?api_key=vxL1eP4u4cKRBQ4PkanKxxBs791wndEQzjxm8lwQ

In [3]:


# Example: U.S. crude oil production
us_prod = get_eia_data("PET.MCRFPUS2.M")

print(us_prod.head())


           date  value
1266 1920-01-01   1097
1265 1920-02-01   1145
1264 1920-03-01   1167
1263 1920-04-01   1165
1262 1920-05-01   1181


In [4]:
us_prod

,date,value
1266,1920-01-01,1097
1265,1920-02-01,1145
1264,1920-03-01,1167
1263,1920-04-01,1165
1262,1920-05-01,1181
...,...,...
4,2025-03-01,13453
3,2025-04-01,13466
2,2025-05-01,13447
1,2025-06-01,13533
